In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb

wandb.login(key="79824aa6b958aeebce669281f175fe198eb060dd", relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc


True

In [3]:
wandb.login()

wandb: Currently logged in as: jabroni_ss (dungeon_as_fate). Use `wandb login --relogin` to force relogin


True

In [4]:
import os
import sys
import time
import torch
import torch.nn
import argparse
import wandb
from PIL import Image
import numpy as np
from validate import validate
from data import create_dataloader
from networks.trainer import Trainer
from options.train_options import TrainOptions
from options.test_options import TestOptions
from earlystop import EarlyStopping
import time
import util

# test config
vals = ['progan', 'stylegan', 'stylegan2', 'biggan', 'cyclegan', 'stargan', 'gaugan', 'deepfake']
multiclass = [1, 1, 1, 0, 1, 0, 0, 0]

def get_val_opt():
    val_opt = TrainOptions().parse(print_options=False)
    val_opt.dataroot = '../../CNNDetection_dataset/val'
    val_opt.isTrain = False
    val_opt.no_resize = False
    val_opt.no_crop = False
    val_opt.serial_batches = True
    val_opt.classes = []
    val_opt.jpg_method = ['pil']
    if len(val_opt.blur_sig) == 2:
        b_sig = val_opt.blur_sig
        val_opt.blur_sig = [(b_sig[0] + b_sig[1]) / 2]
    if len(val_opt.jpg_qual) != 1:
        j_qual = val_opt.jpg_qual
        val_opt.jpg_qual = [int((j_qual[0] + j_qual[-1]) / 2)]
    return val_opt

# params
opt = TrainOptions().parse()
opt.name = "test-4class-resnet-car-cat-chair-horse" + time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
# opt.suffix = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
opt.dataroot = '../../CNNDetection_dataset/'
# opt.classes = "airplane,bicycle,bird,boat,bottle,bus,car,cat,chair,cow,diningtable,dog,horse,motorbike,person,pottedplant,sheep,sofa,train,tvmonitor"
opt.classes = ["car","cat","chair","horse"]
opt.blur_prob = 0
opt.blur_sig = 0
opt.jpg_prob = 0
opt.jpg_method = 'cv2'
opt.batch_size = 32
opt.niter = 1
expr_dir = os.path.join(opt.checkpoints_dir, opt.name)
util.mkdirs(expr_dir)


Testdataroot = os.path.join(opt.dataroot, 'test')
opt.dataroot = '{}/{}/'.format(opt.dataroot, opt.train_split)
print('  '.join(list(sys.argv)) )
val_opt = get_val_opt()
Testopt = TestOptions().parse(print_options=False)
data_loader = create_dataloader(opt)


wandb.init(project='CNNDetection_test_start', name=opt.name, config=opt)

model = Trainer(opt)

print(f'Model is running on device: {next(model.model.parameters()).device}')

def testmodel():
    print('*'*25);accs = [];aps = []
    print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
    for v_id, val in enumerate(vals):
        Testopt.dataroot = '{}/{}'.format(Testdataroot, val)
        Testopt.classes = os.listdir(Testopt.dataroot) if multiclass[v_id] else ['']
        # Testopt.no_resize = True
        # Testopt.no_crop = True
        acc, ap, _, _, _, _ = validate(model.model, Testopt)
        accs.append(acc);aps.append(ap)
        print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id, val, acc*100, ap*100))
        wandb.log({f'{val}_acc': acc, f'{val}_ap': ap, 'step': model.total_steps})
    mean_acc = np.array(accs).mean()*100
    mean_ap = np.array(aps).mean()*100
    
    print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id+1,'Mean', np.array(accs).mean()*100, np.array(aps).mean()*100));print('*'*25) 
    print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
    wandb.log({'Mean_acc': mean_acc, 'Mean_ap': mean_ap, 'step': model.total_steps})


# model.eval();testmodel()
model.train()
print(f'cwd: {os.getcwd()}')
early_stopping = EarlyStopping(patience=opt.earlystop_epoch, delta=-0.001, verbose=True)

for epoch in range(opt.niter):
    epoch_start_time = time.time()
    iter_data_time = time.time()
    epoch_iter = 0

    wandb.log({'Epoch': epoch + 1, 'step': model.total_steps})

    for i, data in enumerate(data_loader):
        model.total_steps += 1
        epoch_iter += opt.batch_size

        model.set_input(data)
        model.optimize_parameters()

        if model.total_steps % opt.loss_freq == 0:
            print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()), "Train loss: {} at step: {} lr {}".format(model.loss, model.total_steps, model.lr))
            wandb.log({'Train_loss': model.loss, 'step': model.total_steps})

    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
            (epoch, model.total_steps))
        model.save_networks('latest')
        model.save_networks(epoch)


    # Validation
    model.eval()
    acc, ap = validate(model.model, val_opt)[:2]
    wandb.log({'Val_accuracy': acc, 'Val_ap': ap, 'step': model.total_steps, 'Epoch': epoch + 1})
    print("(Val @ epoch {}) acc: {}; ap: {}".format(epoch, acc, ap))

    early_stopping(acc, model)
    if early_stopping.early_stop:
        cont_train = model.adjust_learning_rate()
        if cont_train:
            print("Learning rate dropped by 10, continue training...")
            early_stopping = EarlyStopping(patience=opt.earlystop_epoch, delta=-0.002, verbose=True)
        else:
            print("Early stopping.")
            break

    testmodel()
    model.train()

model.eval();testmodel()
model.save_networks('last')
wandb.finish()

----------------- Options ---------------
                     arch: res50                         
               batch_size: 64                            
                    beta1: 0.9                           
                blur_prob: 0                             
                 blur_sig: 0.5                           
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
                  classes:                               
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: ./dataset/                    
          earlystop_epoch: 5                             
                    epoch: latest                        
              epoch_count: 1                             
                      fff: /home/jovyan/.local/share/jupyter/runtime/kernel-v2-720346IFd

Model is running on device: cuda:0
cwd: /home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/CNNDetection
2024_08_09_21_58_54 Train loss: 0.042753834277391434 at step: 400 lr 0.0001
2024_08_09_21_59_33 Train loss: 0.4418513774871826 at step: 800 lr 0.0001
2024_08_09_22_00_12 Train loss: 0.0015586024383082986 at step: 1200 lr 0.0001
2024_08_09_22_00_50 Train loss: 0.011782180517911911 at step: 1600 lr 0.0001
2024_08_09_22_01_29 Train loss: 0.010067179799079895 at step: 2000 lr 0.0001
2024_08_09_22_02_07 Train loss: 0.0016716865357011557 at step: 2400 lr 0.0001
2024_08_09_22_02_46 Train loss: 0.002584815490990877 at step: 2800 lr 0.0001
2024_08_09_22_03_24 Train loss: 0.025854848325252533 at step: 3200 lr 0.0001
2024_08_09_22_04_03 Train loss: 0.00014911318430677056 at step: 3600 lr 0.0001
2024_08_09_22_04_42 Train loss: 0.0013748020865023136 at step: 4000 lr 0.0001
2024_08_09_22_05_20 Train loss: 0.01818438433110714 at step: 4400 lr 0.0001
saving the model a

KeyboardInterrupt: 

In [1]:
wandb.finish()

NameError: name 'wandb' is not defined